In [ ]:
from flask import Flask, render_template, request
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

app = Flask(__name__)

# Load pre-trained VGG16 model with weights from ImageNet
model = VGG16(weights="imagenet", include_top=False)

# Function to preprocess image and predict breed
def predict_breed(image_file):
    img = image.load_img(image_file, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)  # Preprocess image (replace with your specific preprocessing steps)
    preds = model.predict(x)
    # Get the predicted class label and confidence score
    predicted_class = decode_predictions(preds, top=1)[0][0][1]
    confidence_score = decode_predictions(preds, top=1)[0][0][2]
    return predicted_class, confidence_score

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/upload", methods=["POST"])
def upload():
    if request.method == "POST":
        # Get uploaded image file
        image_file = request.files["image"]
        if image_file:
            # Predict breed and confidence score
            predicted_breed, confidence_score = predict_breed(image_file)
            return render_template("result.html", image_path=image_file.filename, breed=predicted_breed, score=confidence_score)
        else:
            return render_template("error.html", message="No image uploaded!")

if __name__ == "__main__":
    app.run(debug=True)
